<center><font size=6><b>GEOS 669 Geodetic Methods and Modeling</b></font></center>

## LEAST SQUARES PARAMETER ESTIMATION Example

The code below provides an example for simple least squares approximation in Python. We're giving ourselves a synthetic data set (line), add some noise, set up the matrix and vector components, perform the inversion and analyze the residuals.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

### Forward model of a line

We will investigate the model for a line:

$y = a+bx$

by first giving ourselves some parameter values for $a$ and $b$, sampling this discretely and plotting the model.


In [ ]:
# Forward model of a line y = a+bx
a       = 1
b       = 3.5

x = np.vstack(np.arange(0, 4, 0.1))b
y = np.vstack(a+b*x)

plt.figure()
plt.plot(x, y)
plt.plot(x, y, 'o')
plt.xlabel("x")
plt.ylabel("y")
plt.show()

### Forward model of a line with noise

Now, let's add some noise, our model becomes:

$y_n = a + bx + e$

where $e$ is some noise, here following a normal distribution.


In [ ]:
std_dev = 0.5

noise = np.vstack(np.random.normal(0,std_dev,len(x)))
yn = np.vstack(a+b*x+noise)

print("Clean forward model (line) and noisy observations:")

plt.figure()
plt.plot(x, y)
plt.plot(x, yn, "o")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# Now we can't see the line any more. These are our data.
print("Clean forward model removed ... these are the measured data:")

plt.figure()
plt.plot(x, yn, "o")
plt.xlabel("x")
plt.ylabel("y")
plt.show()



### Guessing a model

Sometimes it seems appropriate to start out with a wild guess. After all, there's some kind of linear relationship in those data. Why not explore this. Key is always to assess the residuals. If there's some systematic trend in the residuals, your model (the parameter values you chose) isn't doing a good job explaining the observations. Occassionally, that's the goal - you want to isolate a signal that requires explanation using a different model. Still, you've gotta assess the residuals.

We're giving ourselves a model guess, $a_{guess}$ and $b_{guess}$, to calculate synthetic observations $y_{guess}$.

In [ ]:
a_guess = 0.0
b_guess = 4.5
y_guess = np.vstack(a_guess + b_guess * x)

print( "Residuals after guessing a model (a=%f, b=%f) ..." % (a_guess, b_guess))

#G      = np.matrix(np.hstack((np.vstack(np.ones(len(x))), x)))
#m      = np.vstack((a_guess,b_guess))
resids = yn - y_guess

plt.figure()
plt.subplot(2,1,1)
plt.plot(x, yn, "o")
plt.plot(x, y_guess)
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(2,1,2)
plt.plot(x, resids, "o")
plt.xlabel("x")
plt.ylabel("residuals")

plt.show()


### Residual Norms

Once we have residuals in vector form, we can look at their norms. Goal of an optimization problem would be to minimize them, that is, find the parameter values which generate forward model values which are generally in good agreement with the data. Two are pretty common. The Euclidian or $L_2$ norm and the Absolute-value or $L_1$ norm:

For residual vector $r$ with $i$ elements:

$$ L_2 = (r_1^2 + r_2^2 + r_3^2 + r_4^2 \dots)^{-1/2} = \sqrt{\sum_{i=1}^{N} r_i^2}$$
$$ L_1 = |r_1| + |r_2| + |r_3| + |r_4| \dots = \sum_{i=1}^{N} |r_i| $$



In [ ]:
print("Respective norms ...")

print("L2-norm:  %f" % ( math.sqrt(sum([v**2   for v in resids])) ))
print("L1-norm:  %f" % ( math.sqrt(sum([abs(v) for v in resids])) ))


The benefit of the $L_1$ norm is robustness against outliers:

In [ ]:
print("Multiply last residual by 100 to simulate large outlier... impact on norms:")

resids[-1] = resids[-1]*100
print("L2-norm:  %f" % ( math.sqrt(sum([v**2   for v in resids])) ))
print("L1-norm:  %f" % ( math.sqrt(sum([abs(v) for v in resids])) ))


### Inversion

Instead of messing around with trial and error through wild guesses, we can just move ahead and employ the normal equations which provide the least squares solution ($L_2$). The details of why this is are outside of the scope of this brief notebook. Checkout GEOS 627 to get the full background on that.

So, the least squares solution, $m_{est}$, through the normal equations is:

$$ m_{est} = (G^T G)^{-1} G^T d$$

where, in our example, $d=y_{noise}=y_n$. We need to set up $G$, which needs to be set up such that the line equation above is adequately represented, meaning this will have two columns, the first constains $1$ and the other the elements of $x$ for equation $i$. There's some operations nexessary to make sure everything is of the correct orientation.

In [ ]:
#set up the system matrix
G = np.matrix(np.hstack( (np.vstack(np.ones(len(x))), x)) )

#inversion
m = (G.T*G).I * G.T * yn
m = m.getA()

print("Model parameter estimates from normal equation solution:")
print("a_est = %.2f (a = %.2f)\nb_est = %.2f (b = %.2f)" % (m[0], a, m[1], b))

### Compare model to data

As a last step, you calculate the forward model $yf$ using the just estimated parameters in $m_{est}$. We can plot the model directly over the data, but also have to look at the residuals, which now are much more scattered looking and don't show much / any substantial systematic trend.

In [ ]:
print("Forward model from estimated model parameters and noisy data ...")

#calculate forward model
yf = m[0]+m[1]*x

plt.figure()
plt.plot(x, yn, "o")
plt.plot(x, yf)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

#calculate residuals and show norms:
resids = yn-yf
print("Residual norms:")
print("L2-norm:  %f" % ( math.sqrt(sum([v**2   for v in resids])) ))
print("L1-norm:  %f" % ( math.sqrt(sum([abs(v) for v in resids])) ))

print("\n\nResiduals ...")
plt.figure()
plt.plot(x, resids, "o")
plt.xlabel("x")
plt.ylabel("residuals")
plt.show()